# Determinación del tamaño de muestra (para estudiantes)
Este cuaderno permite calcular **n** para:
1. Media con σ conocida (z)
2. Media con σ desconocida (t)
3. Proporción p (z)
e incluye **corrección por población finita (CPF)** y **gráficos**.

Usa las calculadoras interactivas y modifica los parámetros.

## Dependencias (ejecuta si es necesario)

In [ ]:
import sys, subprocess
for p in ['numpy','scipy','matplotlib','ipywidgets','pandas']:
    try:
        __import__(p)
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', p])
print('Listo!')

## Funciones de cálculo

In [ ]:
import math
import numpy as np
from scipy import stats

def n_media_sigma_conocida(z, sigma, E):
    return (z*sigma/E)**2

def n_media_sigma_desconocida_iter(s, E, conf=0.95, max_iter=10):
    z = stats.norm.ppf(0.5 + conf/2)
    n = (z*s/E)**2
    for _ in range(max_iter):
        df = max(1, int(round(n))-1)
        t = stats.t.ppf(0.5 + conf/2, df)
        n_new = (t*s/E)**2
        if abs(n_new - n) < 1e-6:
            break
        n = n_new
    return n

def n_proporcion(z, p, E):
    return (z**2 * p*(1-p)) / (E**2)

def correccion_poblacion_finita(n0, N):
    return n0 / (1 + (n0 - 1)/N)

## Calculadoras interactivas

In [ ]:
import ipywidgets as w
import matplotlib.pyplot as plt

def ui_media_sigma_conocida():
    z = w.FloatText(value=1.96, description='z:')
    sigma = w.FloatText(value=12.0, description='σ:')
    E = w.FloatText(value=2.0, description='E:')
    N = w.IntText(value=0, description='N (CPF):')
    out = w.Output()
    def run(_):
        with out:
            out.clear_output()
            n0 = n_media_sigma_conocida(z.value, sigma.value, E.value)
            print(f'n0 = {n0:.2f}  →  ceil = {math.ceil(n0)}')
            if N.value and N.value>0:
                nadj = correccion_poblacion_finita(n0, N.value)
                print(f'CPF (N={N.value}): n_aj = {nadj:.2f}  →  ceil = {math.ceil(nadj)}')
    btn = w.Button(description='Calcular')
    btn.on_click(run)
    display(w.VBox([w.HBox([z, sigma, E, N]), btn, out]))

def ui_media_sigma_desconocida():
    conf = w.FloatSlider(value=0.95, min=0.80, max=0.999, step=0.001, description='Conf:')
    s = w.FloatText(value=13.0, description='s:')
    E = w.FloatText(value=2.0, description='E:')
    N = w.IntText(value=0, description='N (CPF):')
    out = w.Output()
    def run(_):
        with out:
            out.clear_output()
            n0 = n_media_sigma_desconocida_iter(s.value, E.value, conf.value)
            print(f'n (iter t) = {n0:.2f}  →  ceil = {math.ceil(n0)}')
            if N.value and N.value>0:
                nadj = correccion_poblacion_finita(n0, N.value)
                print(f'CPF (N={N.value}): n_aj = {nadj:.2f}  →  ceil = {math.ceil(nadj)}')
    btn = w.Button(description='Calcular')
    btn.on_click(run)
    display(w.VBox([w.HBox([conf, s, E, N]), btn, out]))

def ui_proporcion():
    z = w.FloatText(value=1.96, description='z:')
    p = w.FloatSlider(value=0.30, min=0.01, max=0.99, step=0.01, description='p:')
    E = w.FloatSlider(value=0.05, min=0.005, max=0.20, step=0.005, description='E:')
    N = w.IntText(value=0, description='N (CPF):')
    out = w.Output()
    def run(_):
        with out:
            out.clear_output()
            n0 = n_proporcion(z.value, p.value, E.value)
            print(f'n0 = {n0:.2f}  →  ceil = {math.ceil(n0)}')
            if N.value and N.value>0:
                nadj = correccion_poblacion_finita(n0, N.value)
                print(f'CPF (N={N.value}): n_aj = {nadj:.2f}  →  ceil = {math.ceil(nadj)}')
    btn = w.Button(description='Calcular')
    btn.on_click(run)
    display(w.VBox([w.HBox([z, p, E, N]), btn, out]))

display({'Media σ conocida (z)': ui_media_sigma_conocida,
         'Media σ desconocida (t)': ui_media_sigma_desconocida,
         'Proporción (z)': ui_proporcion})

## Gráficos didácticos (una figura por gráfico)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Gráfico 1: n vs E (media, σ conocida)
z = 1.96; sigma = 12.0
E_vals = np.linspace(0.5, 5.0, 40)
n_vals = (z*sigma/E_vals)**2
plt.figure(figsize=(6,4))
plt.plot(E_vals, n_vals)
plt.xlabel('Margen de error E')
plt.ylabel('n requerido')
plt.title('n vs E (media, σ conocida, z=1.96, σ=12)')
plt.grid(True)
plt.show()

# Gráfico 2: n vs p (proporción, z=1.96, E=0.05)
z = 1.96; E = 0.05
p_vals = np.linspace(0.01, 0.99, 99)
n_vals = (z**2 * p_vals*(1-p_vals)) / (E**2)
plt.figure(figsize=(6,4))
plt.plot(p_vals, n_vals)
plt.xlabel('p')
plt.ylabel('n requerido')
plt.title('n vs p (proporción, z=1.96, E=0.05)')
plt.grid(True)
plt.show()

## Consejos para interpretar resultados
- Con n grande, la diferencia entre usar z y t es pequeña.
- Si no conoces p, usa 0.5 (caso más conservador).
- Considera costos/tiempos y usa CPF si la población es finita.